In [1]:
import torch
import torch_neuronx

In [2]:
torch.__version__

'2.1.2+cu121'

In [3]:
import sys

In [4]:
sys.path.append("../")

In [5]:
# import match_utils as mutils
from lightglue import LightGlue

In [11]:
matcher_cfg = {
                 'name': 'lightglue',
                 'input_dim': 256,
                 'descriptor_dim': 256,
                 'n_layers': 9,
                 'num_heads': 4,
                 'flash': True,
                 'mp': False,
                 'depth_confidence': 0.95,
                 'width_confidence': 0.99,
                 'filter_threshold': 0.1,
              }

In [12]:
matcher = LightGlue(features="disk", **matcher_cfg)
matcher = matcher.eval()

Downloading: "https://github.com/cvg/LightGlue/releases/download/v0.1_arxiv/disk_lightglue.pth" to /home/ubuntu/.cache/torch/hub/checkpoints/disk_lightglue_v0-1_arxiv.pth
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45.4M/45.4M [00:00<00:00, 283MB/s]


In [13]:
import pickle
import io

class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else:
            return super().find_class(module, name)

#contents = pickle.load(f) becomes...
input_features = CPU_Unpickler(open("./lg-input.pkl","rb")).load()

In [14]:
pred = matcher(input_features)

In [15]:
pred

[tensor([[-1, -1, -1,  ..., -1, -1, -1]]),
 tensor([[84, 96, -1,  ..., -1, -1, -1]]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.]], grad_fn=<CopySlices>),
 tensor([[9.6976e-01, 8.9296e-01, 0.0000e+00,  ..., 7.0294e-04, 9.8463e-02,
          3.7209e-05]], grad_fn=<CopySlices>),
 tensor(7),
 tensor([[[   4,   10],
          [  10,   28],
          [  11,   26],
          ...,
          [1702, 2000],
          [1707, 2001],
          [1738, 2003]]]),
 tensor([[0.9636, 0.9931, 0.9956, 0.9301, 0.5737, 0.9851, 0.9873, 0.9719, 0.9744,
          0.9943, 0.9872, 0.4076, 0.6476, 0.9939, 0.9129, 0.9596, 0.9950, 0.9772,
          0.9971, 0.9291, 0.9875, 0.9925, 0.9827, 0.9764, 0.9692, 0.9698, 0.9923,
          0.8318, 0.8930, 0.9660, 0.9365, 0.7401, 0.6620, 0.8455, 0.9108, 0.7441,
          0.9597, 0.9587, 0.7494, 0.9528, 0.9286, 0.9875, 0.9952, 0.9961, 0.1068,
          0.5394, 0.2414, 0.9365, 0.4869, 0.9560, 0.9921, 0.9304, 0.8763, 0.5236,
          0.6264, 0.5472, 0.7242, 0.9974, 0.6551, 0.8399, 0.

In [16]:
model_trace = torch.jit.trace(matcher, input_features)

/home/ubuntu/LightGlue/inf/../lightglue/lightglue.py:488: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert desc0.shape[-1] == self.conf.input_dim
/home/ubuntu/LightGlue/inf/../lightglue/lightglue.py:489: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert desc1.shape[-1] == self.conf.input_dim
/home/ubuntu/LightGlue/inf/../lightglue/lightglue.py:496: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future.

In [17]:
model_trace(input_features)

[tensor([[-1, -1, -1,  ..., -1, -1, -1]]),
 tensor([[84, 96, -1,  ..., -1, -1, -1]]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.]], grad_fn=<CopySlices>),
 tensor([[9.6976e-01, 8.9296e-01, 0.0000e+00,  ..., 7.0294e-04, 9.8463e-02,
          3.7209e-05]], grad_fn=<CopySlices>),
 tensor(7),
 tensor([[[   4,   10],
          [  10,   28],
          [  11,   26],
          ...,
          [1702, 2000],
          [1707, 2001],
          [1738, 2003]]]),
 tensor([[0.9636, 0.9931, 0.9956, 0.9301, 0.5737, 0.9851, 0.9873, 0.9719, 0.9744,
          0.9943, 0.9872, 0.4076, 0.6476, 0.9939, 0.9129, 0.9596, 0.9950, 0.9772,
          0.9971, 0.9291, 0.9875, 0.9925, 0.9827, 0.9764, 0.9692, 0.9698, 0.9923,
          0.8318, 0.8930, 0.9660, 0.9365, 0.7401, 0.6620, 0.8455, 0.9108, 0.7441,
          0.9597, 0.9587, 0.7494, 0.9528, 0.9286, 0.9875, 0.9952, 0.9961, 0.1068,
          0.5394, 0.2414, 0.9365, 0.4869, 0.9560, 0.9921, 0.9304, 0.8763, 0.5236,
          0.6264, 0.5472, 0.7242, 0.9974, 0.6551, 0.8399, 0.

In [ ]:
model_neuron = torch_neuronx.trace(matcher, input_features)

2024-01-23 05:53:05.000289:  5676  INFO ||NEURON_CACHE||: Compile cache path: /var/tmp/neuron-compile-cache
2024-01-23 05:53:05.000300:  5676  INFO ||NEURON_CC_WRAPPER||: Using a cached neff at /var/tmp/neuron-compile-cache/neuronxcc-2.12.54.0+f631c2365/MODULE_2108354257835340671+d41d8cd9/model.neff. Exiting with a successfully compiled graph.
2024-01-23 05:53:05.000565:  5676  INFO ||NEURON_CACHE||: Compile cache path: /var/tmp/neuron-compile-cache
2024-01-23 05:53:05.000574:  5676  INFO ||NEURON_CC_WRAPPER||: Using a cached neff at /var/tmp/neuron-compile-cache/neuronxcc-2.12.54.0+f631c2365/MODULE_8221219792448796712+d41d8cd9/model.neff. Exiting with a successfully compiled graph.
2024-01-23 05:53:05.000794:  5676  INFO ||NEURON_CACHE||: Compile cache path: /var/tmp/neuron-compile-cache
2024-01-23 05:53:05.000802:  5676  INFO ||NEURON_CC_WRAPPER||: Using a cached neff at /var/tmp/neuron-compile-cache/neuronxcc-2.12.54.0+f631c2365/MODULE_5225604489184339909+d41d8cd9/model.neff. Exitin